# TP Final Integrador Visión por Computadora II - CEIA - FIUBA - Cohorte 16

Alumnos: Fabricio Lopretto (a1616) y Santiago José Olaciregui (a1611)

## Balance de Clases

Objetivo: Se busca conocer si las clases (números de las cartas) se encuentran balanceadas.

In [1]:
# Importa librerías necesarias
import cv2
import os
import numpy as np
from collections import Counter
import shutil
import random

In [2]:
# Directorio de la notebook
script_path = os.getcwd()

In [3]:
# Directorio de los archivos de anotaciones (.txt) del conjunto train
annotations_folder_train = script_path + '/data/data_original/train/labels/'

# Lista para almacenar todas las clases detectadas en los archivos
all_classes = []

# Recorre todos los archivos de etiquetas en el directorio
for filename in os.listdir(annotations_folder_train):
    if filename.endswith('.txt'):
        filepath = os.path.join(annotations_folder_train, filename)
        
        # Abre el archivo de anotación y lee las líneas
        with open(filepath, 'r') as file:
            lines = file.readlines()
            for line in lines:
                # Cada línea empieza con el número de clase
                class_id = line.split()[0]
                all_classes.append(int(class_id))

# Contar la cantidad de instancias de cada clase
class_counts = Counter(all_classes)

# Mostrar la cantidad de imágenes por clase
print("Cantidad de imágenes por clase:")
for class_id in range(15):
    print(f"Clase {class_id}: {class_counts[class_id]} imágenes")

print('*'*20)

# Calcular y mostrar si hay desbalance de clases
total_images = sum(class_counts.values())
for class_id, count in class_counts.items():
    percentage = (count / total_images) * 100
    print(f"Clase {class_id}: {count} imágenes ({percentage:.2f}%)")

Cantidad de imágenes por clase:
Clase 0: 1255 imágenes
Clase 1: 1220 imágenes
Clase 2: 1258 imágenes
Clase 3: 1267 imágenes
Clase 4: 1273 imágenes
Clase 5: 1262 imágenes
Clase 6: 1280 imágenes
Clase 7: 1289 imágenes
Clase 8: 1313 imágenes
Clase 9: 1220 imágenes
Clase 10: 1322 imágenes
Clase 11: 1208 imágenes
Clase 12: 1241 imágenes
Clase 13: 1210 imágenes
Clase 14: 1267 imágenes
********************
Clase 7: 1289 imágenes (6.83%)
Clase 8: 1313 imágenes (6.95%)
Clase 12: 1241 imágenes (6.57%)
Clase 9: 1220 imágenes (6.46%)
Clase 10: 1322 imágenes (7.00%)
Clase 14: 1267 imágenes (6.71%)
Clase 4: 1273 imágenes (6.74%)
Clase 2: 1258 imágenes (6.66%)
Clase 1: 1220 imágenes (6.46%)
Clase 3: 1267 imágenes (6.71%)
Clase 13: 1210 imágenes (6.41%)
Clase 5: 1262 imágenes (6.68%)
Clase 6: 1280 imágenes (6.78%)
Clase 0: 1255 imágenes (6.65%)
Clase 11: 1208 imágenes (6.40%)


## Data Augumentation

### Objetivos:

Indagar la necesidad y experimentos para aumentar la cantidad de datos, realizando las siguientes operaciones:

**Transformaciones geométricas:**

1. Rotación: Girar la imagen en ángulos aleatorios.
2. Escalado: Aumentar o reducir el tamaño de la imagen.
3. Translación: Desplazar la imagen horizontal o verticalmente.

**Background augmentation:**

4. Background generation: Enseñar al modelo a distinguir entre fondo y objeto.

**Transformaciones de ruido:**

5. Ruido gaussiano: Añadir ruido aleatorio a la imagen.
6. Ruido sal y pimienta: Introducir píxeles aleatorios en la imagen.

### 1 - Rotación

*Notas*:
1. Toma un recorte de 104x104 píxeles de la esquina superior izquierda de la imagen original y lo repita como mosaico para llenar una imagen de fondo de 416x416 píxeles.
2. Luego, insertaremos el objeto rotado de forma aleatoria sobre el fondo de mosaico.
3. Para cada rotación (90, 180, 270 grados), el script guarda la imagen con el objeto rotado sobre el fondo de mosaico en el directorio de salida y el nuevo etiquetado.

In [ ]:
input_images = script_path + '/data/data_procesada/train_mask_background/images/'
input_annotations = script_path + '/data/data_procesada/train_mask_background/train/labels/'
output_images = script_path + '/data/data_procesada/train_mask_background/images/'
output_annotations = script_path + '/data/data_procesada/train_mask_background/labels/'

# Tamaño de la imagen de salida y del recorte
output_size = (416, 416)
patch_size = (104, 104)

# Función para generar un fondo de mosaico y superponer el objeto rotado
def generate_mosaic_background(image):
    """
    Extrae un recorte de 104x104 píxeles de la esquina superior izquierda de la imagen original.
    Crea una nueva imagen de 416x416 píxeles y llena la imagen con el recorte en forma de mosaico.
    """
    # Extraer un recorte de 104x104 de la esquina superior izquierda
    patch = image[:patch_size[1], :patch_size[0]]
    
    # Crear una imagen de fondo vacía de 416x416 píxeles
    background_image = np.zeros((output_size[1], output_size[0], 3), dtype=np.uint8)
    
    # Repetir el recorte en mosaico para llenar la imagen de fondo
    for y in range(0, output_size[1], patch_size[1]):
        for x in range(0, output_size[0], patch_size[0]):
            end_x = min(x + patch_size[0], output_size[0])
            end_y = min(y + patch_size[1], output_size[1])
            background_image[y:end_y, x:end_x] = patch[:end_y - y, :end_x - x]
    
    return background_image

# Función para rotar el objeto y colocarlo sobre el fondo de mosaico
def rotate_object(image, bbox, angle):
    """
    Rota el objeto extraído de la imagen en el ángulo especificado (90, 180 o 270 grados).
    Genera un fondo de mosaico usando generate_mosaic_background.
    Calcula la posición y dimensiones normalizadas del objeto rotado para actualizar la anotación y lo coloca encima del fondo de mosaico.
    """
    height, width = image.shape[:2]
    x_center, y_center, bbox_width, bbox_height = bbox
    
    # Convertir las dimensiones del bounding box a píxeles
    bbox_width_px = int(bbox_width * width)
    bbox_height_px = int(bbox_height * height)
    
    # Extraer el recorte del objeto
    x1 = max(0, int(x_center * width) - bbox_width_px // 2)
    y1 = max(0, int(y_center * height) - bbox_height_px // 2)
    x2 = x1 + bbox_width_px
    y2 = y1 + bbox_height_px
    object_crop = image[y1:y2, x1:x2]
    
    # Rotar el recorte del objeto
    if angle == 90:
        rotated_object = cv2.rotate(object_crop, cv2.ROTATE_90_CLOCKWISE)
    elif angle == 180:
        rotated_object = cv2.rotate(object_crop, cv2.ROTATE_180)
    elif angle == 270:
        rotated_object = cv2.rotate(object_crop, cv2.ROTATE_90_COUNTERCLOCKWISE)
    else:
        rotated_object = object_crop  # Sin rotación (caso no debería suceder)
    
    # Generar el fondo de mosaico
    background_image = generate_mosaic_background(image)
    
    # Calcular el tamaño del recorte rotado
    rotated_height, rotated_width = rotated_object.shape[:2]
    
    # Calcular nuevas coordenadas para centrar el objeto rotado en el fondo de mosaico
    new_x1 = max(0, int(x_center * output_size[0]) - rotated_width // 2)
    new_y1 = max(0, int(y_center * output_size[1]) - rotated_height // 2)
    new_x2 = new_x1 + rotated_width
    new_y2 = new_y1 + rotated_height
    
    # Insertar el objeto rotado sobre el fondo de mosaico
    background_image[new_y1:new_y2, new_x1:new_x2] = rotated_object
    
    # Calcular nuevas dimensiones normalizadas del bounding box rotado
    new_bbox_width = rotated_width / output_size[0]
    new_bbox_height = rotated_height / output_size[1]
    
    return background_image, (x_center, y_center, new_bbox_width, new_bbox_height)

# Procesar cada imagen y su anotación
for filename in os.listdir(input_images):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(input_images, filename)
        annotation_path = os.path.join(input_annotations, filename.replace('.jpg', '.txt').replace('.png', '.txt'))
        
        # Cargar la imagen y la anotación
        image = cv2.imread(image_path)
        if image is None:
            continue
        
        with open(annotation_path, 'r') as f:
            lines = f.readlines()
        
        for line in lines:
            parts = line.strip().split()
            class_id = int(parts[0])
            bbox = [float(p) for p in parts[1:]]
            
            # Crear imágenes con el objeto rotado en 90, 180 y 270 grados
            for angle in [90, 180, 270]:
                modified_image, new_bbox = rotate_object(image, bbox, angle)
                
                # Guardar la nueva imagen
                angle_suffix = f"rot_{angle}"
                new_image_name = f"{filename.split('.')[0]}_{angle_suffix}.jpg"
                new_image_path = os.path.join(output_images, new_image_name)
                cv2.imwrite(new_image_path, modified_image)
                
                # Guardar la nueva anotación
                new_annotation_name = f"{filename.split('.')[0]}_{angle_suffix}.txt"
                new_annotation_path = os.path.join(output_annotations, new_annotation_name)
                
                with open(new_annotation_path, 'w') as f_out:
                    f_out.write(f"{class_id} {new_bbox[0]} {new_bbox[1]} {new_bbox[2]} {new_bbox[3]}\n")

print("Proceso completado. Las imágenes y anotaciones rotadas con fondo de mosaico se han guardado.")

Proceso completado. Las imágenes y anotaciones rotadas se han guardado.


### 2 - Escalado

Se generan imagenes con objetos ampliados o reducidos a partir de las imagenes preexistentes.

*Notas:*
1. Toma un cuadrado de 104x104 píxeles de la esquina superior izquierda de la imagen original y lo replicará en mosaico para crear un fondo de 416x416 píxeles.
2. El objeto escalado se colocará sobre este fondo de mosaico.
3. Para cada factor de escala (scale_factors), el script guarda la imagen con el objeto redimensionado sobre el fondo de mosaico en el directorio de salida y su etiquetado.

In [ ]:
# Directorios de entrada y salida
input_images = script_path + '/data/data_procesada/train_mask_background/images/'
input_annotations = script_path + '/data/data_procesada/train_mask_background/labels/'
output_images = script_path + '/data/data_procesada/train_mask_background/images/'
output_annotations = script_path + '/data/data_procesada/train_mask_background/labels/'

# Dimensiones de la imagen de salida y del recorte
output_size = (416, 416)
patch_size = (104, 104)

# Factor de escalado para aumentar o reducir el tamaño del objeto
scale_factors = [0.5, 0.75, 1.25, 1.5]

# Función para generar un fondo de mosaico
def generate_mosaic_background(image):
    """
    Extrae un recorte de 104x104 píxeles de la esquina superior izquierda de la imagen original.
    Crea una nueva imagen de fondo de 416x416 píxeles y llena la imagen con el recorte en forma de mosaico.
    """
    # Extraer un recorte de 104x104 de la esquina superior izquierda
    patch = image[:patch_size[1], :patch_size[0]]
    
    # Crear una imagen de fondo vacía de 416x416 píxeles
    background_image = np.zeros((output_size[1], output_size[0], 3), dtype=np.uint8)
    
    # Repetir el recorte en mosaico para llenar la imagen de fondo
    for y in range(0, output_size[1], patch_size[1]):
        for x in range(0, output_size[0], patch_size[0]):
            end_x = min(x + patch_size[0], output_size[0])
            end_y = min(y + patch_size[1], output_size[1])
            background_image[y:end_y, x:end_x] = patch[:end_y - y, :end_x - x]
    
    return background_image

# Función para escalar el objeto y colocarlo sobre el fondo de mosaico
def resize_object(image, bbox, scale_factor):
    """
    Redimensiona el objeto extraído de la imagen en función del scale_factor.
    Genera un fondo de mosaico usando generate_mosaic_background.
    Calcula la posición y dimensiones normalizadas del objeto redimensionado
    para actualizar la anotación y lo coloca encima del fondo de mosaico.
    """
    height, width = image.shape[:2]
    x_center, y_center, bbox_width, bbox_height = bbox
    
    # Convertir las dimensiones del bounding box a píxeles
    bbox_width_px = int(bbox_width * width)
    bbox_height_px = int(bbox_height * height)
    
    # Calcular las nuevas dimensiones del bounding box
    new_bbox_width_px = int(bbox_width_px * scale_factor)
    new_bbox_height_px = int(bbox_height_px * scale_factor)
    
    # Asegurarse de que el nuevo tamaño no exceda los límites de la imagen
    if new_bbox_width_px > output_size[0] or new_bbox_height_px > output_size[1]:
        new_bbox_width_px = min(new_bbox_width_px, output_size[0])
        new_bbox_height_px = min(new_bbox_height_px, output_size[1])
    
    # Extraer el recorte del objeto original
    x1 = max(0, int(x_center * width) - bbox_width_px // 2)
    y1 = max(0, int(y_center * height) - bbox_height_px // 2)
    x2 = x1 + bbox_width_px
    y2 = y1 + bbox_height_px
    object_crop = image[y1:y2, x1:x2]
    
    # Redimensionar el recorte del objeto
    resized_object = cv2.resize(object_crop, (new_bbox_width_px, new_bbox_height_px), interpolation=cv2.INTER_LINEAR)
    
    # Generar el fondo de mosaico
    background_image = generate_mosaic_background(image)
    
    # Calcular las coordenadas del nuevo centro en píxeles
    new_x1 = max(0, int(x_center * output_size[0]) - new_bbox_width_px // 2)
    new_y1 = max(0, int(y_center * output_size[1]) - new_bbox_height_px // 2)
    new_x2 = new_x1 + new_bbox_width_px
    new_y2 = new_y1 + new_bbox_height_px
    
    # Colocar el objeto redimensionado en el fondo de mosaico
    background_image[new_y1:new_y2, new_x1:new_x2] = resized_object
    
    # Calcular las nuevas dimensiones normalizadas
    new_bbox_width = new_bbox_width_px / output_size[0]
    new_bbox_height = new_bbox_height_px / output_size[1]
    
    return background_image, (x_center, y_center, new_bbox_width, new_bbox_height)

# Procesar cada imagen y su anotación
for filename in os.listdir(input_images):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(input_images, filename)
        annotation_path = os.path.join(input_annotations, filename.replace('.jpg', '.txt').replace('.png', '.txt'))
        
        # Cargar la imagen y la anotación
        image = cv2.imread(image_path)
        if image is None:
            continue
        
        with open(annotation_path, 'r') as f:
            lines = f.readlines()
        
        for line in lines:
            parts = line.strip().split()
            class_id = int(parts[0])
            bbox = [float(p) for p in parts[1:]]
            
            # Crear imágenes con el objeto en diferentes tamaños
            for scale_factor in scale_factors:
                modified_image, new_bbox = resize_object(image, bbox, scale_factor)
                
                # Guardar la nueva imagen
                scale_suffix = f"scale_{scale_factor}"
                new_image_name = f"{filename.split('.')[0]}_{scale_suffix}.jpg"
                new_image_path = os.path.join(output_images, new_image_name)
                cv2.imwrite(new_image_path, modified_image)
                
                # Guardar la nueva anotación
                new_annotation_name = f"{filename.split('.')[0]}_{scale_suffix}.txt"
                new_annotation_path = os.path.join(output_annotations, new_annotation_name)
                
                with open(new_annotation_path, 'w') as f_out:
                    f_out.write(f"{class_id} {new_bbox[0]} {new_bbox[1]} {new_bbox[2]} {new_bbox[3]}\n")

print("Proceso completado. Las imágenes y anotaciones ajustadas se han guardado.")

Proceso completado. Las imágenes y anotaciones ajustadas se han guardado.


### 3 - Traslación

*Notas:*

1. Toma un recorte de 104x104 píxeles de la esquina superior izquierda de la imagen original y lo replicará como un mosaico para crear un fondo de 416x416 píxeles.
2. El objeto reubicado se coloca sobre este fondo de mosaico.
3. Para cada posición especificada, genera una nueva imagen con el objeto reposicionado de forma aleatoria sobre el fondo de mosaico.
4. Guarda las imágenes modificadas y sus anotaciones actualizadas en los directorios de salida y su etiqueta.

In [ ]:
# Rutas a los directorios de imágenes y anotaciones
input_images = script_path + '/data/data_procesada/train_mask_background/images/'
input_annotations = script_path + '/data/data_procesada/train_mask_background/labels/'
output_images = script_path + '/data/data_procesada/train_mask_background/images/'
output_annotations = script_path + '/data/data_procesada/train_mask_background/labels/'

# Dimensiones de la imagen de salida y del recorte
output_size = (416, 416)
patch_size = (104, 104)

# Función para generar un fondo de mosaico
def generate_mosaic_background(image):
    # Extraer un recorte de 104x104 de la esquina superior izquierda
    patch = image[:patch_size[1], :patch_size[0]]
    
    # Crear una imagen de fondo vacía de 416x416 píxeles
    background_image = np.zeros((output_size[1], output_size[0], 3), dtype=np.uint8)
    
    # Repetir el recorte en mosaico para llenar la imagen de fondo
    for y in range(0, output_size[1], patch_size[1]):
        for x in range(0, output_size[0], patch_size[0]):
            end_x = min(x + patch_size[0], output_size[0])
            end_y = min(y + patch_size[1], output_size[1])
            background_image[y:end_y, x:end_x] = patch[:end_y - y, :end_x - x]
    
    return background_image

# Función para reubicar el objeto y colocarlo sobre el fondo de mosaico
def relocate_object(image, bbox):
    """
    Calcula el recorte del objeto de acuerdo con su bounding box.
    Genera un fondo de mosaico utilizando generate_mosaic_background.
    Coloca el objeto en una de las cuatro posiciones aleatorias
    en el fondo de mosaico.
    """
    height, width = image.shape[:2]
    x_center, y_center, bbox_width, bbox_height = bbox
    
    # Convertir las dimensiones del bounding box a píxeles
    bbox_width_px = int(bbox_width * width)
    bbox_height_px = int(bbox_height * height)
    x_center_px = int(x_center * width)
    y_center_px = int(y_center * height)
    
    # Extraer el recorte del objeto
    x1 = max(0, x_center_px - bbox_width_px // 2)
    y1 = max(0, y_center_px - bbox_height_px // 2)
    x2 = min(width, x_center_px + bbox_width_px // 2)
    y2 = min(height, y_center_px + bbox_height_px // 2)
    
    object_crop = image[y1:y2, x1:x2]
    
    # Generar el fondo de mosaico
    background = generate_mosaic_background(image)
    
    # Generar cuatro posiciones aleatorias para el centro del objeto
    random_positions = []
    for _ in range(4):
        new_x_center_px = random.randint(bbox_width_px // 2, output_size[0] - bbox_width_px // 2)
        new_y_center_px = random.randint(bbox_height_px // 2, output_size[1] - bbox_height_px // 2)
        random_positions.append((new_x_center_px, new_y_center_px))
    
    # Procesar cada posición aleatoria
    modified_images_and_bboxes = []
    for i, (new_x_center_px, new_y_center_px) in enumerate(random_positions):
        # Calcular las nuevas coordenadas de recorte del objeto en la imagen de fondo
        new_x1 = max(0, new_x_center_px - bbox_width_px // 2)
        new_y1 = max(0, new_y_center_px - bbox_height_px // 2)
        
        # Colocar el recorte del objeto en la posición aleatoria sobre el fondo de mosaico
        modified_background = background.copy()
        modified_background[new_y1:new_y1 + object_crop.shape[0], new_x1:new_x1 + object_crop.shape[1]] = object_crop
        
        # Normalizar las coordenadas del centro del objeto en la imagen de fondo
        new_x_center = new_x_center_px / output_size[0]
        new_y_center = new_y_center_px / output_size[1]
        
        modified_images_and_bboxes.append((modified_background, (new_x_center, new_y_center, bbox_width, bbox_height)))
    
    return modified_images_and_bboxes

# Procesar cada imagen y su anotación
for filename in os.listdir(input_images):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(input_images, filename)
        annotation_path = os.path.join(input_annotations, filename.replace('.jpg', '.txt').replace('.png', '.txt'))

        # Cargar la imagen y la anotación
        image = cv2.imread(image_path)
        if image is None:
            continue
        
        with open(annotation_path, 'r') as f:
            lines = f.readlines()
        
        for line in lines:
            parts = line.strip().split()
            class_id = int(parts[0])
            bbox = [float(p) for p in parts[1:]]
            
            # Crear cuatro nuevas imágenes con el objeto en posiciones aleatorias
            modified_images_and_bboxes = relocate_object(image, bbox)
            for i, (modified_image, new_bbox) in enumerate(modified_images_and_bboxes):
                
                # Guardar la nueva imagen
                new_image_name = f"{filename.split('.')[0]}_random_{i}.jpg"
                new_image_path = os.path.join(output_images, new_image_name)
                cv2.imwrite(new_image_path, modified_image)
                
                # Guardar la nueva anotación
                new_annotation_name = f"{filename.split('.')[0]}_random_{i}.txt"
                new_annotation_path = os.path.join(output_annotations, new_annotation_name)
                
                with open(new_annotation_path, 'w') as f_out:
                    f_out.write(f"{class_id} {new_bbox[0]} {new_bbox[1]} {new_bbox[2]} {new_bbox[3]}\n")

print("Proceso completado. Las imágenes y anotaciones modificadas se han guardado.")

Proceso completado. Las imágenes y anotaciones modificadas se han guardado.


### 4 - Background generation: Se generan imagenes sin objetos de ninguna clase.

*Notas:*

Asegurarnos de que las imágenes de fondo tengan las mismas dimensiones que las originales, podemos modificar el script para generar una imagen de fondo completa. La solución implica seleccionar áreas aleatorias sin objetos y copiarlas en diferentes partes de la imagen original hasta llenarla completamente. De esta forma, obtenemos una imagen del mismo tamaño que la original sin ningún objeto visible.

1. Para cada imagen en el dataset, el script genera una versión de fondo con el mismo tamaño y la guarda en el directorio de salida.
2. Asigna un nombre a la nueva imagen que indica que es de fondo (_background).

In [ ]:
# Rutas a los directorios de imágenes y anotaciones
input_images = script_path + '/data/data_procesada/train_mask_background/images/'
input_annotations = script_path + '/data/data_procesada/train_mask_background/labels/'
output_background = script_path + '/data/data_procesada/train_mask_background/images/'

# Dimensiones de la imagen de salida y del recorte
output_size = (416, 416)
patch_size = (104, 104)

# Función para generar una imagen de fondo de tamaño fijo a partir de un recorte de 104x104 de la esquina superior izquierda
def generate_tiled_background(image):
    # Extraer el recorte de la esquina superior izquierda
    patch = image[:patch_size[1], :patch_size[0]]
    
    # Crear una imagen de fondo vacía del tamaño deseado
    background_image = np.zeros((output_size[1], output_size[0], 3), dtype=np.uint8)
    
    # Repetir el recorte en mosaico para llenar la imagen de fondo
    for y in range(0, output_size[1], patch_size[1]):
        for x in range(0, output_size[0], patch_size[0]):
            # Pegar el recorte en la posición actual, ajustando si se sale de los límites
            end_x = min(x + patch_size[0], output_size[0])
            end_y = min(y + patch_size[1], output_size[1])
            background_image[y:end_y, x:end_x] = patch[:end_y - y, :end_x - x]
    
    return background_image

# Genera la misma cantidad de fondos solos que imagenes con on¡bjetos para balancear
for i in range(52):
    # Procesar cada imagen
    for filename in os.listdir(input_images):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(input_images, filename)
            
            # Cargar la imagen
            image = cv2.imread(image_path)
            if image is None:
                continue
            
            # Generar la imagen de fondo
            background_image = generate_tiled_background(image)
            
            # Guardar la imagen de fondo
            new_image_name = f"{filename.split('.')[0]}_background.jpg"
            new_image_path = os.path.join(output_background, new_image_name)
            cv2.imwrite(new_image_path, background_image)

print("Proceso completado. Las imágenes de fondo en mosaico se han guardado.")

Proceso completado. Las imágenes de fondo en mosaico se han guardado.


### 5 - Generación de ruido

*Nota:*

Agrega ruido Gaussiano o ruido "sal y pimienta" de forma aleatoria a cada imagen en tu dataset y guarda las imágenes con ruido aplicando uno de estos dos tipos de ruido en cada imagen.

1. Para cada imagen, el script elige aleatoriamente entre ruido gaussiano y ruido "sal y pimienta".
2. Guardado de las imágenes y anotaciones. La imagen con ruido se guarda en el directorio de salida con un sufijo que indica el tipo de ruido aplicado (_gaussian o _salt_pepper).
3. Las anotaciones se copian sin cambios al nuevo nombre, ya que el ruido no afecta la ubicación o tamaño del objeto.

In [ ]:
# Rutas a los directorios de imágenes y anotaciones
input_images = script_path + '/data/data_procesada/train_mask_background/images/'
input_annotations = script_path + '/data/data_procesada/train_mask_background/labels/'
output_images = script_path + '/data/data_procesada/train_mask_background/images/'
output_annotations = script_path + '/data/data_procesada/train_mask_background/labels/'

# Función para agregar ruido Gaussiano
def add_gaussian_noise(image, mean=0, sigma=25):
    """
    Agrega ruido gaussiano con una media y desviación estándar configurables.
    Se aplica usando una distribución normal, y la imagen resultante se limita a valores entre 0 y 255.
    """
    gaussian_noise = np.random.normal(mean, sigma, image.shape).astype(np.float32)
    noisy_image = cv2.add(image.astype(np.float32), gaussian_noise)
    return np.clip(noisy_image, 0, 255).astype(np.uint8)

# Función para agregar ruido Sal y Pimienta
def add_salt_and_pepper_noise(image, salt_prob=0.01, pepper_prob=0.01):
    """
    Agrega ruido "sal y pimienta".
    La "sal" (blanco) y la "pimienta" (negro) se aplican en proporciones
    aleatorias de píxeles definidos por salt_prob y pepper_prob.
    """
    noisy_image = np.copy(image)
    # Sal (blanco)
    num_salt = np.ceil(salt_prob * image.size)
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape[:2]]
    noisy_image[coords[0], coords[1]] = 255

    # Pimienta (negro)
    num_pepper = np.ceil(pepper_prob * image.size)
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape[:2]]
    noisy_image[coords[0], coords[1]] = 0
    return noisy_image

# Procesar cada imagen y su anotación
for filename in os.listdir(input_images):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(input_images, filename)
        annotation_path = os.path.join(input_annotations, filename.replace('.jpg', '.txt').replace('.png', '.txt'))
        
        # Copiar la anotación original al directorio de salida
        shutil.copy(annotation_path, output_annotations)
        
        # Cargar la imagen
        image = cv2.imread(image_path)
        if image is None:
            continue
        
        # Decidir aleatoriamente el tipo de ruido
        noise_type = random.choice(['gaussian', 'salt_and_pepper'])
        
        # Agregar el ruido seleccionado a la imagen
        if noise_type == 'gaussian':
            noisy_image = add_gaussian_noise(image)
            noise_suffix = 'gaussian'
        else:
            noisy_image = add_salt_and_pepper_noise(image)
            noise_suffix = 'salt_pepper'
        
        # Guardar la imagen con ruido
        new_image_name = f"{filename.split('.')[0]}_{noise_suffix}.jpg"
        new_image_path = os.path.join(output_images, new_image_name)
        cv2.imwrite(new_image_path, noisy_image)
        
        # Copiar el archivo de anotación original con el nuevo nombre
        new_annotation_name = f"{filename.split('.')[0]}_{noise_suffix}.txt"
        new_annotation_path = os.path.join(output_annotations, new_annotation_name)
        shutil.copy(annotation_path, new_annotation_path)

print("Proceso completado. Las imágenes con ruido y anotaciones se han guardado.")

Proceso completado. Las imágenes con ruido y anotaciones se han guardado.


### Observaciones:

Al analizar el balance de clases, resultó que el conjunto de datos se encuentra balanceado por defecto. Con lo cual no es necesario aumentar la cantidad de imagenes con objetos de una clase en particular. Lo único que se consideró necesario fue generar imagenes solo con fondo, dado que en el conjunto de imagenes utilizado, todas las imagenes contienen una carta (y por ende objeto).

Por otro lado, se evaluo incorporar imagenes aplicando transformaciones geométricas y ruido a las existentes. Al realizar esto, el tamaño del conjunto de datos resultó inmanejable para los recursos disponibles. Por este motivo, se decidió utilizar arquitecturas como YOLO que disponen de un *pipeline* con *data augumentation* durante el entrenamiento por lotes. En la correspondiente notebook, se explora esta opción.